In [1]:
import os
import PyPDF2
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\Rideema
[nltk_data]     Malji\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Rideema
[nltk_data]     Malji\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [79]:
wnl = WordNetLemmatizer()

In [3]:
# Define the directory containing the PDF files
directory = 'C:\\Users\\Rideema Malji\\OneDrive\\Desktop\\Others\\Upwork\\data labeling\\annual_reports_III\\annual_reports_III\\'

# Define the list of keywords to search for
candidates_scope1 = 'Direct emissions, on-site emissions, on site emissions, combustion emissions, fuel consumption, stationary sources, fugitive emissions, process emissions, greenhouse gas emissions, carbon footprint, Clean Air Act, emissions reductions, carbon tax, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope1 = candidates_scope1.split(', ')

candidates_scope2 = 'Indirect emissions, purchased electricity, grid-purchased electricity, energy consumption, greenhouse gas emissions, carbon footprint, Renewable Energy Certificates (RECs), power purchase agreements (PPAs), energy efficiency, demand-side management, energy conservation, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope2 = candidates_scope2.split(', ')

candidates_scope3 = 'Indirect emissions, supply chain emissions, upstream emissions, downstream emissions, transportation emissions, distribution emissions, use phase emissions, end-of-life emissions, embodied carbon, life cycle assessment (LCA), carbon footprint, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope3 = candidates_scope3.split(', ')

In [5]:
# Cleans the sentences and gives you clean things to search for in pdfs

def clean_keyword_sentences(candidates):
    keywords_scope = []
    # We get every phrase from the keywords list
    for phrase in candidates:
        # each keyword from the keywords list will be lemmatized
        keywords = []
        for keyword in phrase.split():
            keywords.append(wnl.lemmatize(keyword.lower()))
        keywords_scope.append(' '.join(keywords))
        
    return keywords_scope

In [11]:
keywords_scope1 = clean_keyword_sentences(candidates_scope1)

In [80]:
keywords_scope1

['direct emission',
 'on-site emission',
 'on site emission',
 'combustion emission',
 'fuel consumption',
 'stationary source',
 'fugitive emission',
 'process emission',
 'greenhouse gas emission',
 'carbon footprint',
 'clean air act',
 'emission reduction',
 'carbon tax',
 'carbon offsetting',
 'carbon credit',
 'carbon trading',
 'carbon footprint reduction',
 'carbon neutral',
 'carbon management',
 'carbon accounting']

 # Preprocessing on the given dataset for extracting the logical relationship between the sentences/words in given corpus

In [168]:
# Iterate over each file in the directory

# We input which scope we are looking for, it uses the correct search keyword list for this specific scope 
scope=1
if scope == 1:
    search_for_list = candidates_scope1
elif scope == 2:
    search_for_list = candidates_scope2
elif scope == 3:
    search_for_list = candidates_scope3
else:
    print('Scope should be 1,2 or 3')

keywords = clean_keyword_sentences(search_for_list)

examples_to_annotate = []
scope = scope

counter = 1
content=[]

for filename in os.listdir(directory):
  
    if filename.endswith('.pdf'):
      try:
        # Open the PDF file
        with open(os.path.join(directory, filename), 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            # Iterate over each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
    
                # Get the sentences by splitting by '.'
                sentences = text.split('.')
            
                
                
                # Clean every sentence by removing newline characters
                sentences_cleaned = [' '.join(text.split()) for text in sentences]

                # lemmatize the words
                sentences_lemmatized = [' '.join([wnl.lemmatize(keyword.lower()) for keyword in text.split()]) for text in sentences_cleaned]

                # Iterate over each keyword
                #for keyword in keywords:
                #    for i in range(len(sentences_lemmatized)):
                    
                    # Search for keyword in the text
                #      if keyword in sentences_lemmatized[i]:
                          #print(f'{keyword} found on page {page_num+1} of {filename}')
                          # we define the dictionary item to add to our list
                          # We take sentences_cleaned's i th item to append
                          # We search by using the lemmatized sentences, but we take the normal cleaned versions
                item = {'filename': filename,
                        'page': page_num,
                        'sentence': sentences_cleaned,
                        'scope': scope,
                        'annotation': ''}
                content.extend(sentences)
                examples_to_annotate.append(item)
        counter=counter+1
        print('Total pdf count is ===>',counter)
        if counter==12:
            break


      except Exception as e:
        print(e)


Total pdf count is ===> 2
Total pdf count is ===> 3
Total pdf count is ===> 4
Total pdf count is ===> 5
Total pdf count is ===> 6
Total pdf count is ===> 7
Total pdf count is ===> 8
Total pdf count is ===> 9
Total pdf count is ===> 10
Total pdf count is ===> 11
Total pdf count is ===> 12


In [169]:
import pandas as pd
df = pd.DataFrame.from_dict(examples_to_annotate)

In [175]:
from gensim.utils import simple_preprocess
content=list(set(content))
tokenized =[]
for sentence in content:
  # the simple_preprocess function returns a list of each sentence
  tokenized.append(simple_preprocess(sentence, deacc = True))
print(tokenized)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [177]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
  
stop_words = set(stopwords.words('english'))

filtered_toknzd=[]
for tk in tokenized:
    
# converts the words in word_tokens to lower case and then checks whether 
#they are present in stop_words or not
    filtered_sentence = [w for w in tk if not w.lower() in stop_words]
    filtered_toknzd.append(filtered_sentence)

In [178]:
tokenized=filtered_toknzd.copy()

In [181]:
tokenized

[[],
 ['november', 'related', 'financial', 'liability', 'amounting'],
 ['castello',
  'agogna',
  'pv',
  'fiac',
  'professional',
  'air',
  'compressors',
  'bologna',
  'fiac'],
 ['total', 'emitted', 'approximately', 'metric', 'tons', 'co', 'equivalents'],
 ['merck',
  'kg',
  'accordance',
  'article',
  'articles',
  'association',
  'well',
  'separate',
  'combined',
  'non',
  'financial',
  'group',
  'report'],
 [],
 ['figures',
  'refer',
  'people',
  'directly',
  'employed',
  'merck',
  'therefore',
  'may',
  'deviate',
  'figures',
  'financial',
  'section',
  'report'],
 ['acquisitions'],
 ['accounting',
  'leases',
  'ifrs',
  'group',
  'leases',
  'predominantly',
  'relate',
  'offices',
  'malta',
  'territories',
  'also',
  'relate',
  'small',
  'items',
  'equipme',
  'nt'],
 ['million',
  'gilead',
  'related',
  'recently',
  'renegotiated',
  'agreement',
  'december',
  'filgotinib'],
 ['depending',
  'performance',
  'key',
  'performance',
  'indicato

In [185]:
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
 
# extract a list of words from the dataset
data =[]
for word in tokenized:
  data.append(word)

print('The length of data is =====>',data)
# We will split the data into two parts
data_1 = data[:int(len(data)/0.8)]   # this is used to train the model
data_2 = data[int(len(data)/0.8):]   # this part will be used to update the model
 
# Training the Word2Vec model
w2v_model = Word2Vec(data_1, min_count = 0, workers = cpu_count())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [193]:
keyword_1=[]
for keywrd1 in candidates_scope1:
    keyword_1.extend(keywrd1.split(' '))

In [209]:
final_key1=[]
for key1 in keyword_1:
    try:
        kwords,proba=zip(*w2v_model.wv.most_similar(key1,topn=8))## we can use scope 1 ,2,3 here
    except:
        print('keyword ',key1,' not found in the dict')
    final_key1.extend(kwords)
final_key1=list(set(final_key1))

keyword  Direct  not found in the dict
keyword  on-site  not found in the dict
keyword  on  not found in the dict
keyword  Clean  not found in the dict
keyword  Air  not found in the dict
keyword  Act  not found in the dict


In [214]:
train_df=df[['sentence']][:int(len(df)*0.8)]
validate_df=df[['sentence']][:int(len(df)*0.8)]

In [227]:
sum([1 if x in ['Electrolux', 'Annual', 'Report', '2019','greenhouse'] else 0 for x in final_key1])

1

In [232]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model.label_model import LabelModel

ABSTAIN = -1
GREENHOUSE = 1
NON_GREENHOUSE = 0


@labeling_function()
def lenth_sentence(record):
    print(record['sentence'][0].split(' '))
    try:
        if len(record['sentence'][0].split(' '))>5:
            if sum([1 if x in record['sentence'][0].split(' ') else 0 for x in final_key1])<5:
                return NON_GREENHOUSE
            else:
                print('here')
                return GREENHOUSE
        else:
            return NON_GREENHOUSE
    except:
        return NON_GREENHOUSE
    
lfs = [lenth_sentence]
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_df)
L_validate = applier.apply(df=validate_df)

 12%|█████████▍                                                                    | 191/1576 [00:00<00:01, 956.82it/s]

['Electrolux', 'Annual', 'Report', '2019']
['Well', 'positioned', 'to', 'create', 'value', 'A', 'strong', 'focus', 'on', 'innovation', 'to', 'improve', 'the', 'consumer', 'experience', 'and', 'a', 'track', 'record', 'of', 'successfully', 'driving', 'cost', 'efficiency', 'are', 'important', 'competitive', 'assets']
['']
['']
['CEO', 'STATEMENT', 'Strengthening', 'our', 'innovation', 'and', 'efficiency', 'platform', '5', 'Business', 'overview', '6', 'Key', 'areas', 'to', 'create', 'value', 'and', 'drive', 'profitable', 'growth', '10', 'Well-positioned', 'in', 'a', 'changing', 'world', '12', 'Celebrating', 'a', 'century', 'of', 'better', 'living', '14', 'REPORTING', 'Report', 'by', 'the', 'Board', 'of', 'Directors', '17', 'Notes', '44', 'Proposed', 'distribution', 'of', 'earnings', '81', 'Auditors’', 'report', '82', 'Eleven-year', 'review', '86', 'Operations', 'by', 'business', 'area,', 'yearly', '88', 'Quarterly', 'information', '89', 'Sustainability', 'reporting', '90', 'Corporate', 'go

 26%|████████████████████▌                                                         | 416/1576 [00:00<00:01, 991.29it/s]

['Cont']
['Cont']
['Note', '23', 'Other', 'provisions', 'Group', 'Parent', 'Company', 'Provisions', 'for', 'restructuring', 'Warranty', 'commitments', 'Claims', 'Other', 'TotalProvisions', 'for', 'restructuringWarranty', 'commitments', 'Other', 'Total', 'Opening', 'balance,', 'January', '1,', '2019', '1,210', '2,095', '1,397', '2,863', '7,565', '174', '473', '44', '691', 'Acquisitions', 'of', 'operations', '—', '5', '—', '—', '5', '—', '—', '—', '—', 'Provisions', 'made', '1,326', '2,178', '421', '1,733', '5,658', '471', '370', '8', '849', 'Provisions', 'used', '–564', '–1,984', '–491', '–1,258', '–4,297', '–87', '–396', '–14', '–497', 'Unused', 'amounts', 'reversed', '–179', '–31', '—', '–56', '–266', '–23', '—', '—', '–23', 'Exchange-rate', 'differences', '22', '68', '50', '70', '210', '–1', '5', '—', '4', 'Discontinued', 'operations', '–86', '–245', '—', '–361', '–692', '0', '—', '0', '0', 'Closing', 'balance,', 'December', '31,', '2019', '1,729', '2,086', '1,377', '2,991', '8,183',

 39%|██████████████████████████████▌                                               | 617/1576 [00:00<00:01, 896.66it/s]

...


['176', 'Corporate', 'Governance', '___', 'Statement', 'on', 'Corporate', 'Governance', 'including', 'Compensation', 'Report', 'Supervisory', 'Board', 'The', 'Supervisory', 'Board', 'has', '16', 'members']
['177', 'Corporate', 'Governance', '___', 'Statement', 'on', 'Corporate', 'Governance', 'including', 'Compensation', 'Report', 'The', 'Supervisory', 'Board', 'performs', 'a', 'monitoring', 'function']
['178', 'Corporate', 'Governance', '___', 'Statement', 'on', 'Corporate', 'Governance', 'including', 'Compensation', 'Report', 'in', 'the', 'Supervisory', 'Board', '’s', 'profile', 'of', 'skills', 'and', 'expertise,', 'which', 'stipulates', 'that', 'the', 'Supervisory', 'Board', 'have', 'at', 'least', 'four', 'members', 'who', 'possess', 'such', 'knowledge', 'of', 'the', 'sector']
['179', 'Corporate', 'Governance', '___', 'Statement', 'on', 'Corporate', 'Governance', 'including', 'Compensation', 'Report', 'The', 'Board', 'of', 'Partners', 'supervises', 'the', 'Executive', 'Board', 'in'

 93%|███████████████████████████████████████████████████████████████████████▎     | 1460/1576 [00:01<00:00, 758.21it/s]


['Introduction', 'to', 'governance', 'The', 'Group', 'has', 'three', 'decision-making', 'bodies', 'in', 'a', 'hierarchical', 'relationship', 'to', 'one', 'another:', 'the', 'Shareholders’', 'meeting,', 'the', 'Board', 'of', 'Directors', 'and', 'the', 'Chief', 'Executive', 'Officer']
['Board', 'of', 'Directors', 'An', 'experienced', 'team,', 'committed', 'to', 'high', 'standards', 'of', 'corporate', 'governance']
['7XIJER\x040YRHFSVK', 'Board', 'member,', "'LEMVQER\x04SJ\x04XLI\x04Remuneration", 'Committee', '7[IHMWL\x04GMXM^IR\x12\x04&SVR\x04\x15', '\x1a\x19\x12\x04', '&SEVH\x04QIQFIV\x04WMRGI\x04\x16\x14\x15\x14\x12\x04-RHITIRHIRX\x12', 'Current', 'assignments', '&SEVH\x04QIQFIV\x04SJ\x04+\x19\x04)RXIVXEMRQIRX\x04%&\x04', '4YFP', "\x10\x047EJMVEF\x04%&\x04ERH\x0403<\x04'SRXEMRIV\x048IGLRSPSK]\x04%&\x12\x04'LEMVQER\x04SJ\x04XLI\x04&SEVH\x04EX\x04)ROPEVI\x04)OSRSQM\x047ZIVMKI\x04%&\x04ERH\x042SXI\x04(IWMKR\x047XYHMS\x12", 'Holdings', '\x19\x17\x1b\x10\x1a\x14\x14\x04/MRHVIH\x04+VSYT\x0

100%|█████████████████████████████████████████████████████████████████████████████| 1576/1576 [00:01<00:00, 806.30it/s]



['XS\x04E\x04VEXI\x04SJ\x047)/\x04\x15\x15\x12\x15\x1a\x15\x04TIV\x04+&4', '\x12\x048LIWI\x04QSZIQIRXW\x04MR\x04WSQI\x04', 'SJ\x04XLI\x04+VSYTƅW\x04TVMRGMTEP\x04XVEHMRK\x04GYVVIRGMIW\x04GSRXVMFYXIH\x04XS\x04XLI\x04SZIVEPPɸJSVIMKR\x04I\\GLERKI\x04PSWW\x04SR\x04STIVEXMSRW\x04EW\x04WLS[R\x04MR\x04RSXI\x04\x18\x04SRɸTEKI\x04\x1b\x18\x12\x04', '%HHMXMSREP\x04JSVIMKR\x04I\\GLERKI\x04HMWGPSWYVIW\x04JSV\x04GYWXSQIV\x04FEPERGIW\x04', 'EVI\x04GSRXEMRIH\x04MR\x04RSXI\x04\x15', '\x04SR\x04TEKI\x04', '\x17\x12', '8LI\x04+VSYT\x04EPWS\x04LEW\x04RIX\x04MRZIWXQIRXW\x04MR\x04JSVIMKR\x04STIVEXMSRW\x10\x04', '[LSWI\x04RIX\x04EWWIXW\x04EVI\x04I\\TSWIH\x04XS\x04JSVIMKR\x04GYVVIRG]\x04XVERWPEXMSR\x04VMWO\x12\x04-R\x04EHHMXMSR\x10\x04XLI\x04+VSYT\x04VITSVXW\x04MR\x04+&4\x04WXIVPMRK\x10\x04[LMGL\x04MW\x04', 'XLI\x04', 'GYVVIRG]\x04MR\x04[LMGL\x04MXW\x04S[R\x04WLEVI\x04GETMXEP\x04MW\x04HIRSQMREXIH\x10\x04EPXLSYK', 'L\x04', 'MX\x04MW\x04MRGSVTSVEXIH\x04ERH\x04XVEHMRK\x04MR\x041EPXE\x12', '%W\x04IPIQIRXW\x04SJ\

In [237]:
flatten_list=L_train.flatten(order='C')

In [240]:
train_df['Annotate']=flatten_list

In [242]:
train_df.to_csv('Final_annotation_training.csv')